In [1]:
import json
import random
from pathlib import Path
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, Subset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AdamW
from tqdm import tqdm

In [2]:
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()

In [3]:
%%time
pretrained_dir = "pretrained/google/electra-small-discriminator"
model_max_length = 512
tokenizer = AutoTokenizer.from_pretrained(pretrained_dir, model_max_length=model_max_length)
print(f"{repr(tokenizer)}\n{tokenizer.model_input_names}")

PreTrainedTokenizerFast(name_or_path='pretrained/google/electra-small-discriminator', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})
['input_ids', 'token_type_ids', 'attention_mask']
CPU times: user 15.6 ms, sys: 15.6 ms, total: 31.2 ms
Wall time: 29.3 ms


In [4]:
train = pd.read_parquet("input/squad/train.parquet")
train.drop(index=train[train["id"] == "5acd29f507355d001abf3774"].index, inplace=True)
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 130318 entries, 0 to 130318
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             130318 non-null  object
 1   is_impossible  130318 non-null  int8  
 2   question       130318 non-null  object
 3   answer_start   130318 non-null  int16 
 4   answer_end     130318 non-null  int16 
 5   answer_text    130318 non-null  object
 6   context        130318 non-null  object
dtypes: int16(2), int8(1), object(4)
memory usage: 5.6+ MB


In [9]:
%%time
enc = tokenizer(list(train["context"]), list(train["question"]))
print(f"{repr(enc.keys())}\nlen={len(enc['input_ids'])}")

Token indices sequence length is longer than the specified maximum sequence length for this model (518 > 512). Running this sequence through the model will result in indexing errors


dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
len=130318
CPU times: user 53.1 s, sys: 4.97 s, total: 58 s
Wall time: 15.1 s


In [10]:
%%time
indices = []
for i, v in enumerate(enc["input_ids"]):
    if len(v) <= model_max_length:
        indices.append(i)
train = train.iloc[indices]
enc = tokenizer(list(train["context"]), list(train["question"]), padding="max_length", return_offsets_mapping=True)
enc.update({"id": list(train["id"])})
print(f"{repr(enc.keys())}\nlen={len(enc['input_ids'])}")

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'id'])
len=130134
CPU times: user 1min 11s, sys: 30.1 s, total: 1min 41s
Wall time: 45 s


In [11]:
def add_token_positions(encodings, answer_start, answer_end):
    start_positions = []
    end_positions = []
    for i in range(len(answer_start)):
        j = encodings.char_to_token(i, answer_start[i])
        if j is None:
            offsets = encodings["offset_mapping"][i]
            _id = encodings["id"][i]
            raise ValueError(f"start pos must not be None. i={i}, id={_id}, answer_start={answer_start[i]}\noffsets={offsets}") 
        k = encodings.char_to_token(i, answer_end[i] - 1)
        if k is None:
            raise ValueError("end pos must not be None")
        if j > k:
            raise ValueError("start pos must be less than or equals end pos")
        start_positions.append(j)
        end_positions.append(k)
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

In [12]:
class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)


In [13]:
%%time
add_token_positions(enc, list(train["answer_start"]), list(train["answer_end"]))
del enc["id"]
train_ds = SquadDataset(enc)

CPU times: user 250 ms, sys: 0 ns, total: 250 ms
Wall time: 261 ms


In [14]:
%%time
model = AutoModelForQuestionAnswering.from_pretrained(pretrained_dir)
print(repr(model.config))

Some weights of the model checkpoint at pretrained/google/electra-small-discriminator were not used when initializing ElectraForQuestionAnswering: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at pretrained/google/electra-small-discriminator and are newly initialized: ['qa_outpu

ElectraConfig {
  "_name_or_path": "pretrained/google/electra-small-discriminator",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 4,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "transformers_version": "4.5.1",
  "type_vocab_size": 2,
  "vocab_size": 30522
}

CPU times: user 375 ms, sys: 15.6 ms, total: 391 ms
Wall time: 319 ms


In [15]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
model.train()
max_examples = 2 * 128
indices = range(len(train_ds))
indices = random.sample(indices, max_examples)
sample_ds = Subset(train_ds, indices)
train_loader = DataLoader(sample_ds, batch_size=128, shuffle=True)
#train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
optim = AdamW(model.parameters(), lr=5e-4)
epochs = 2
for epoch in range(epochs):
    loss_mean = 0
    steps = len(train_loader)
    for batch in tqdm(train_loader):
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch["token_type_ids"].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)
        outputs = model(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask,
                        start_positions=start_positions, end_positions=end_positions)
        loss = outputs[0]
        loss_mean += loss / steps
        loss.backward()
        optim.step()
    print(f"epoch={epoch}, loss={loss_mean:.4f}")

  0%|          | 0/2 [00:00<?, ?it/s]

epoch=0, loss=5.8852


100%|██████████| 2/2 [06:30<00:00, 195.18s/it]

epoch=1, loss=5.0955


In [16]:
model.eval()

ElectraForQuestionAnswering(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (embeddings_project): Linear(in_features=128, out_features=256, bias=True)
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_featu

In [17]:
model.save_pretrained("tmp")